In [1]:
from lxml import etree
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from matplotlib.patches import Polygon
import os
from bs4 import BeautifulSoup
import re
from collections import Counter
import pandas as pd

In [2]:
path = '/home/noe/Documents/2022-2023/ENC/Mémoire/Viard/Viard.xml'
viard = '/home/noe/Documents/2022-2023/ENC/Mémoire/Viard/'

In [3]:
#création d'un xml avec juste les textes sans l'apparat
with open(path,'r') as file:
    data = file.read()
soup = BeautifulSoup(data,'xml')
tag = soup.find('group')
new_soup = BeautifulSoup(features='xml')
new_soup.append(tag)
with open('group.xml', 'w') as f:
    f.write(new_soup.prettify())

In [4]:
element = 'notum facimus'
element_2 = 'Notum facimus'

In [5]:
with open(viard+'group.xml','r') as file:
    data = file.read()

In [6]:
#création des fichiers pour chaque texte
soup = BeautifulSoup(data,'xml')
text_tags = soup.find_all('text')
for i, tag in enumerate(text_tags):
    new_soup = BeautifulSoup(features='xml')
    new_soup.append(tag)
    with open(f'text_{i}.xml', 'w') as f:
        f.write(new_soup.prettify())

In [7]:
path = '/home/noe/Documents/2022-2023/ENC/Mémoire/Viard/textes/'
os.chdir(path)

In [19]:
i = 0

dic = {}

while i <= 443:
    
    with open(f'text_{i}.xml','r') as file:
        data = file.read()
    
    latin = []
    key_name = []
    
    if element in data or element_2 in data:
        
        soup = BeautifulSoup(data,'xml')
        for tag in soup.find_all(final):
            tag.decompose()
        
        body_tag = soup.find('body')
        body_text = body_tag.text.strip()
        latin.append(body_text)
        key_name.append(f'text_{i+1}')

    dic.update({str(key_name):latin})
    
    i+=1

In [20]:
#get list of tags

i = 0

liste_tags = []

while i <=443:
    
    with open(f'text_{i}.xml','r') as file:
        data = file.read()
        
    if element in data or element_2 in data:
        soup=BeautifulSoup(data,'xml')
        parent = soup.find('body')
        tag_list = [tag.name for tag in parent.find_all()]
        liste_tags.extend(tag_list)
        
    i += 1
#cration of final list
final = list(set(liste_tags))

final.remove('p')

In [21]:
keys_to_remove = []

for key, value in dic.items():
    if value == []:
        keys_to_remove.append(key)

for key in keys_to_remove:
    del dic[key]

In [22]:
colnames = ['id','text']
df = pd.DataFrame(dic.items())
df.columns = colnames

In [23]:
def clean_string(s):
    s = s.replace("['", "")
    s = s.replace("']", "")
    return s

In [24]:
df['id'] = df['id'].apply(clean_string)

In [25]:
df['text'] = df['text'].apply(lambda x: ','.join(map(str, x)))

In [29]:
def clean_text(string):
    first = re.sub("\n","",string)
    new_string = re.sub("[0-9]. ",'',first)
    new_string_2 = re.sub('- ','',new_string)
    final = re.sub(' . ',' ', new_string_2)
    final2 = re.sub('-, ','',final)
    final3 = re.sub('[0-9]','',final2)
    return final3

In [33]:
def clean_column(text):
    text = text.strip()
    text = re.sub(r"\s+", " ", text)
    return text

In [30]:
df['text'] = df['text'].apply(clean_text)

In [34]:
df['text'] = df['text'].apply(clean_column)

In [38]:
from typing import List
from pie_extended.cli.utils import get_tagger, get_model, download

# In case you need to download
do_download = False
if do_download:
    for dl in download("lasla"):
        x = 1

# model_path allows you to override the model loaded by another .tar
model_name = "lasla"
tagger = get_tagger(model_name, batch_size=256, device="cpu", model_path=None)



i = 0
while i <= 64:
    
    liste_lemmes = []
    
    sentences: List[str] = [df['text'][i]]
    # Get the main object from the model (: data iterator + postprocesor
    from pie_extended.models.lasla.imports import get_iterator_and_processor
    for sentence_group in sentences:
        iterator, processor = get_iterator_and_processor()
        lemma = tagger.tag_str(sentence_group, iterator=iterator, processor=processor) 

    for elm in lemma:
        if elm['pos'] == 'ADJqua' or elm['pos'] == 'VER' or elm['pos'] == 'NOMcom' or elm['pos'] == 'NOMpro' or elm['pos'] == 'ADJcar' or elm['pos'] == 'ADJord' or elm['pos'] == 'ADJmul' or elm['pos'] == 'ADJdis':
            liste_lemmes.append(elm['lemma'])
    texte = ' '.join(liste_lemmes)
    
    with open(f'texte_{i+1}.txt','w') as file:
        file.write(texte)
    i+=1

KeyError: 65

In [48]:
i = 0
lemmas = []

while i < 65:
    with open(f'texte_{i+1}.txt','r') as file:
        data = file.readlines()
    lemmas.extend(data)
    i+=1

In [50]:
df['lemmas'] = lemmas

In [53]:
df['text'][0]

'Philippus, Valesii et Andegavie comes, Francie et Navarre regna regens, notum facimus universis presentibus et futuris quod cum Stephanus le Haubergier, civis Parisiensis, quoddam nobile feodum valoris duodecim librarum parisiensium annui redditus, vel circa, situm in villa de Antogniaco et eius territorio, quod religioso viro, abbate Sancti Germani de Pratis prope Parisius tenetur per homagium, certo et legitimo titulo, sicut asserit, duxerit acquirendum; prefatus tamen abbas, eumdem Stephanum, eo quod nobilis non existit, in suum recipere homagium pro memorato feodo recusavit et adhuc recu sat; unde, nobis, prefatus Stephanus humiliter supplicavit ut super hoc sibi specialem gratiam facere dignaremur. Nos igitur, ejusdem in hac parte supplicationibus favorabiliter annuentes, eidem Stephano, de speciali gratia concedimus, pro se et heredibus suis, ut ipse et ipsius heredes, licet idem Stephanus, a nobilibus originem non traxerit, dictum feodum, ac si esset nobilis tenere possint impe